# Single Photon Counting Test

## Imports

In [1]:
import h5py
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pycentroids as centroids

ImportError: /home/swilkins/miniconda3/lib/python3.7/site-packages/pycentroids-0.0.0-py3.7-linux-x86_64.egg/pycentroids.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _Z19find_photons_uint16PtS_PdS0_mmti

In [ ]:
centroids.__version__

In [ ]:
help(centroids.find_photons)

## Load test data

In [ ]:
image = h5py.File('../../Centroid_Test/scan_52912/image040.h5', 'r')
xip = pd.read_csv('../../Centroid_Test/scan_52912/image040-xray.txt', delim_whitespace=True)

In [ ]:
data = np.asarray(image['data'])

## Call the fast code for single photon counting

In [ ]:
table, out, bias = centroids.find_photons(data, 400, 2)

In [ ]:
df = pd.DataFrame(table)

## Plot the results 
### Whole image

In [ ]:
fig, ax = plt.subplots(3,1)
ax[0].imshow(data, vmin=150, vmax=300)
ax[1].imshow(out>0)
ax[2].imshow(bias)

## Selected part of the image

In [ ]:
fig, ax = plt.subplots(3,1)
ax[0].imshow(data[850:900,600:700], vmin=150, vmax=300)
ax[1].imshow(out[850:900,600:700]>0)
ax[2].imshow(bias[850:900,600:700] )

In [ ]:
df.sort_values(by=[0, 1])

In [ ]:
xip.sort_values(by=['x', 'y'])

In [ ]:
fig, ax = plt.subplots(1,1)
for row in table:
    ax.plot(row[8:])
ax.set_ylabel('ADUs')
ax.set_xlabel('Pixel number')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.hist(table[:,7:17].sum(axis=1), bins=25, range=(800, 2600))
ax.set_ylabel('N')
ax.set_xlabel('ADUs')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.hist(table[:,18:].sum(axis=1), bins=20, range=(-200, 200))
ax.set_ylabel('N')
ax.set_xlabel('ADUs')